In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix

In [69]:
train_data = pd.read_csv('marketing_sample_for_walmart_com-walmart_com_product_review__20200701_20201231__5k_data.tsv', sep='\t')

In [70]:
train_data = train_data[['Uniq Id','Product Id', 'Product Rating', 'Product Reviews Count', 'Product Category', 'Product Brand', 'Product Name', 'Product Image Url', 'Product Description', 'Product Tags']]

In [71]:
train_data.head(5)

,Uniq Id,Product Id,Product Rating,Product Reviews Count,Product Category,Product Brand,Product Name,Product Image Url,Product Description,Product Tags
0,1705736792d82aa2f2d3caf1c07c53f4,2e17bf4acecdece67fc00f07ad62c910,NaN,NaN,Premium Beauty > Premium Makeup > Premium Nail...,OPI,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,NaN,"OPI Infinite Shine, Nail Lacquer Nail Polish, ..."
1,95a9fe6f4810fcfc7ff244fd06784f11,076e5854a62dd283c253d6bae415af1f,NaN,NaN,Beauty > Hair Care > Hair Color > Auburn Hair ...,Nice'n Easy,"Nice n Easy Permanent Color, 111 Natural Mediu...",https://i5.walmartimages.com/asr/9c8e42e4-13a5...,Pack of 3 Pack of 3 for the UPC: 381519000201 ...,"Nice 'n Easy Permanent Color, 111 Natural Medi..."
2,8d4d0330178d3ed181b15a4102b287f2,8a4fe5d9c7a6ed26cc44d785a454b124,4.5,29221.0,Beauty > Hair Care > Hair Color > Permanent Ha...,Clairol,Clairol Nice N Easy Permanent Color 7/106A Nat...,https://i5.walmartimages.com/asr/e3a601c2-6a2b...,This Clairol Nice N Easy Permanent Color gives...,Clairol Nice 'N Easy Permanent Color 7/106A Na...
3,fddc4df45b35efd886794b261f730c51,03b5fb878a33eadff8b033419eab9669,NaN,NaN,Beauty > Makeup > Lip,Kokie Cosmetics,"Kokie Professional Matte Lipstick, Hot Berry, ...",https://i5.walmartimages.com/asr/25b4b467-bc61...,Calling all matte lip lovers! Indulge in our r...,"Kokie Professional Matte Lipstick, Hot Berry, ..."
4,0990cf89a59ca6a0460349a3e4f51d42,ce3d761e57d6ccad80619297b5b1bcbc,NaN,131.0,Seasonal > Stock Up Essentials > Personal Care...,Gillette,"Gillette TRAC II Plus Razor Blade Refills, Fit...",https://i5.walmartimages.com/asr/1a2ebb06-cd01...,"In 1971, Gillette introduced the Trac II razor...","Gillette TRAC II Plus Razor Blade Refills, Fit..."


In [72]:
train_data.isnull().sum()

Uniq Id                     0
Product Id                  0
Product Rating           2806
Product Reviews Count    1654
Product Category           10
Product Brand              13
Product Name                0
Product Image Url           0
Product Description      1127
Product Tags                0
dtype: int64

In [73]:
train_data = train_data[['Uniq Id', 'Product Id', 'Product Rating', 'Product Reviews Count', 'Product Category', 'Product Brand', 'Product Name', 'Product Image Url', 'Product Description', 'Product Tags']]
train_data['Product Rating'].fillna(0, inplace=True)
train_data['Product Reviews Count'].fillna(0, inplace=True)
train_data['Product Category'].fillna('', inplace=True)
train_data['Product Brand'].fillna('', inplace=True)
train_data['Product Description'].fillna('', inplace=True)

In [74]:
train_data.isnull().sum()

Uniq Id                  0
Product Id               0
Product Rating           0
Product Reviews Count    0
Product Category         0
Product Brand            0
Product Name             0
Product Image Url        0
Product Description      0
Product Tags             0
dtype: int64

In [75]:
train_data.duplicated().sum()

0

In [76]:
column_name_mapping = {
    'Uniq Id': 'ID', 'Product Id': 'ProdID', 'Product Rating': 'Rating', 'Product Reviews Count': 'ReviewCount',
    'Product Category': 'Category', 'Product Brand': 'Brand', 'Product Name': 'Name', 'Product Image Url': 'ImageURL',
    'Product Description': 'Description', 'Product Tags': 'Tags'
}
train_data.rename(columns=column_name_mapping, inplace=True)

In [77]:
train_data['ID'] = train_data['ID'].str.extract(r'(\d+)').astype(float)
train_data['ProdID'] = train_data['ProdID'].str.extract(r'(\d+)').astype(float)

In [78]:
average_ratings = train_data.groupby(['Name','ReviewCount','Brand','ImageURL'])['Rating'].mean().reset_index()

In [79]:
top_rated_items = average_ratings.sort_values(by='Rating', ascending=False)

rating_base_recommendation = top_rated_items.head(10)

In [80]:
rating_base_recommendation['Rating'] = rating_base_recommendation['Rating'].astype(int)
rating_base_recommendation['ReviewCount'] = rating_base_recommendation['ReviewCount'].astype(int)

C:\Users\gopik\AppData\Local\Temp\ipykernel_12192\565725879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_base_recommendation['Rating'] = rating_base_recommendation['Rating'].astype(int)
C:\Users\gopik\AppData\Local\Temp\ipykernel_12192\565725879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_base_recommendation['ReviewCount'] = rating_base_recommendation['ReviewCount'].astype(int)


In [81]:
print("Rating Base Recommendation System: (Trending Products)")

rating_base_recommendation

Rating Base Recommendation System: (Trending Products)


,Name,ReviewCount,Brand,ImageURL,Rating
1686,"Electric Shaver, Triple Shaving Time Electric ...",4,Moosoo,https://i5.walmartimages.com/asr/e7dcd553-90df...,5
526,"Alaffia Body Lotion, Vanilla, 32 Oz",2,Alaffia,https://i5.walmartimages.com/asr/2988c323-cb6f...,5
2053,"Gold Bond Ultimate Ultimate Healing Lotion, Al...",2,Gold Bond,https://i5.walmartimages.com/asr/34b610e7-05db...,5
4716,"Versace Man Eau Fraiche Eau De Toilette Spray,...",24,Versace,https://i5.walmartimages.com/asr/edaaeed5-9da0...,5
2058,Goldwell StyleSign 1 Flat Marvel Straightening...,2,Goldwell,https://i5.walmartimages.com/asr/3bf90289-6980...,5
3842,"Red Devil 0322 Steel Wool # 00 Very Fine, 8 Pa...",1,Red Devil,https://i5.walmartimages.com/asr/60bfe5ba-774c...,5
510,"Air Wick Plug in Starter Kit, Warmer + 1 Refil...",1,Air Wick,https://i5.walmartimages.com/asr/0fac65b2-c6aa...,5
3841,Recovery Complex Anti-Frizz Shine Serum by Bai...,4,Bain de Terre,https://i5.walmartimages.com/asr/fcdb4d2e-3727...,5
2687,Long Aid Extra Dry Formula Curl Activator Gel ...,12,Long Aid,https://i5.walmartimages.com/asr/f7f29199-bfa5...,5
2062,"Good Sense 60-Day Air Care System, Citrus, 2 o...",1,Diversey,https://i5.walmartimages.com/asr/025a7068-7bb1...,5


In [82]:
def reduce_tfidf_dimensionality(tfidf_matrix, n_components=100):
    svd = TruncatedSVD(n_components=n_components)
    tfidf_reduced = svd.fit_transform(tfidf_matrix)
    return tfidf_reduced

In [83]:
def content_based_recommendations(train_data, item_name, top_n=10):
    if item_name not in train_data['Name'].values:
        print(f"Item '{item_name}' not found in the training data.")
        return pd.DataFrame()

    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix_content = tfidf_vectorizer.fit_transform(train_data['Tags'])
    tfidf_reduced_content = reduce_tfidf_dimensionality(tfidf_matrix_content)
    cosine_similarities_content = cosine_similarity(tfidf_matrix_content, tfidf_matrix_content)
    item_index = train_data[train_data['Name'] == item_name].index[0]
    similar_items = list(enumerate(cosine_similarities_content[item_index]))
    similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)
    top_similar_items = similar_items[1:top_n+1]
    recommended_item_indices = [x[0] for x in top_similar_items]
    recommended_items_details = train_data.iloc[recommended_item_indices][['Name', 'ReviewCount', 'Brand', 'ImageURL', 'Rating']]
    return recommended_items_details

In [84]:
item_name = 'OPI Infinite Shine, Nail Lacquer Nail Polish, Bubble Bath'
content_based_rec = content_based_recommendations(train_data, item_name, top_n=8)

content_based_rec

,Name,ReviewCount,Brand,ImageURL,Rating
4854,"OPI Infinite Shine Nail Polish, Dulce De Leche...",1.0,OPI,https://i5.walmartimages.com/asr/466527fe-9e5e...,5.0
3052,"OPI Nail Lacquer Nail Polish, Hawaiian Orchid",1.0,OPI,https://i5.walmartimages.com/asr/cef5a3ca-dfed...,4.0
4010,"OPI Nail Polish, Are We There Yet?, 0.5 Fl Oz",2.0,OPI,https://i5.walmartimages.com/asr/bb0c5e20-0ead...,3.0
4752,"OPI Infinite Shine Nail Polish, Let Love Spark...",3.0,OPI,https://i5.walmartimages.com/asr/7eef48ac-3a28...,4.0
4356,"OPI Nail Lacquer Nail Polish, Go with the Lava...",1.0,OPI,https://i5.walmartimages.com/asr/6cd23e3d-f3be...,5.0
3967,"OPI Infinite Shine Nail Polish, Mini Scotland ...",1.0,OPI,https://i5.walmartimages.com/asr/b673df9d-cc73...,0.0
2952,"OPI Nail Lacquer Nail Polish, Miami Beet",0.0,OPI,https://i5.walmartimages.com/asr/083f9170-da65...,0.0
4966,OPI Nail GelColor + Infinite Shine Polish -TAU...,0.0,OPI,https://i5.walmartimages.com/asr/a364a6d7-0cd1...,0.0


In [85]:
item_name = 'Kokie Professional Matte Lipstick, Hot Berry, 0.14 fl oz'
content_based_rec = content_based_recommendations(train_data, item_name, top_n=8)

content_based_rec

,Name,ReviewCount,Brand,ImageURL,Rating
546,"Kokie Professional Matte Lipstick, Kiss Me, 0....",0.0,Kokie Cosmetics,https://i5.walmartimages.com/asr/27dd82a2-2b9c...,0.0
3406,"Kokie Professional Matte Lipstick, Firecracker...",0.0,Kokie Cosmetics,https://i5.walmartimages.com/asr/8312221b-ed22...,0.0
2873,Kokie Professional Lip Poudre Liquid Matte Liq...,7.0,Kokie Cosmetics,https://i5.walmartimages.com/asr/31c99d9b-ea11...,3.4
4050,Kokie Professional Lip Poudre Liquid Matte Liq...,7.0,Kokie Cosmetics,https://i5.walmartimages.com/asr/fdd7498c-319f...,3.4
3023,Be Matte Lipstick - Pink,2.0,City Color,https://i5.walmartimages.com/asr/4425a13e-085f...,3.0
558,"Kokie Professional Nail Polish, Girls Night , ...",6.0,Kokie Cosmetics,https://i5.walmartimages.com/asr/a6f6da7a-cb11...,4.2
4920,"Kokie Professional Nail Polish, Jamacan Me Blu...",6.0,Kokie Cosmetics,https://i5.walmartimages.com/asr/16fc0e60-1c66...,4.2
3620,"Kokie Professional Skin Perfect HD Foundation,...",9.0,Kokie Cosmetics,https://i5.walmartimages.com/asr/2f8fcaa9-7db7...,0.0


In [86]:
def collaborative_filtering_recommendations(train_data, target_user_id, top_n=10):
    user_item_matrix = train_data.pivot_table(index='ID', columns='ProdID', values='Rating', aggfunc='mean').fillna(0)
    
    # Matrix Factorization (NMF)
    nmf = NMF(n_components=20)
    W = nmf.fit_transform(user_item_matrix)
    H = nmf.components_

    target_user_index = user_item_matrix.index.get_loc(target_user_id)
    user_similarities = cosine_similarity(W[target_user_index].reshape(1, -1), W)
    
    similar_users_indices = user_similarities.argsort()[0][::-1][1:]
    recommended_items = []
    for user_index in similar_users_indices:
        rated_by_similar_user = user_item_matrix.iloc[user_index]
        not_rated_by_target_user = (rated_by_similar_user > 0) & (user_item_matrix.iloc[target_user_index] == 0)
        recommended_items.extend(user_item_matrix.columns[not_rated_by_target_user])

    recommended_items_details = train_data[train_data['ProdID'].isin(recommended_items)][['Name', 'ReviewCount', 'Brand', 'ImageURL', 'Rating']]
    return recommended_items_details.head(10)

target_user_id = 4
top_n = 5
collaborative_filtering_rec = collaborative_filtering_recommendations(train_data, target_user_id)
print(f"Top {top_n} recommendations for User {target_user_id}:")
collaborative_filtering_rec

C:\Users\gopik\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\gopik\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


Top 5 recommendations for User 4:


,Name,ReviewCount,Brand,ImageURL,Rating
1,"Nice n Easy Permanent Color, 111 Natural Mediu...",0.0,Nice'n Easy,https://i5.walmartimages.com/asr/9c8e42e4-13a5...,0.0
5,Old Spice Artisan Styling High Hold Matte Fini...,52.0,Old Spice,https://i5.walmartimages.com/asr/73214b95-72ce...,4.6
6,"Colgate My First Baby and Toddler Toothpaste, ...",10.0,Colgate,https://i5.walmartimages.com/asr/d28b16f4-1bd4...,4.3
9,(3 pack) Vaseline Lip Therapy Tinted Lip Balm ...,950.0,Vaseline,https://i5.walmartimages.com/asr/6fced36e-5459...,0.0
10,"The Art of Shaving Mens Hair Styling Gel, Juni...",0.0,The Art of Shaving,https://i5.walmartimages.com/asr/661839a6-67ac...,0.0
11,"ORLY Nail Lacquer for Women, #20634 Red Carpet...",2.0,ORLY,https://i5.walmartimages.com/asr/e37d0f7d-defd...,4.5
12,"LOreal Paris True Match Super-Blendable Blush,...",184.0,L'Oreal Paris,https://i5.walmartimages.com/asr/506a78ca-cca6...,3.8
13,"Pantene Shampoo and Conditioner Set, Sheer Vol...",413.0,Pantene,https://i5.walmartimages.com/asr/d13252b0-a730...,4.1
17,Aromatherapy Essential Oil Diffuser 200ml - Ma...,12.0,Magnifeko,https://i5.walmartimages.com/asr/43f7de97-84bc...,4.8
18,"Hard Candy Wet Ever Bold Hold Lip Lacquer Tin,...",38.0,Hard Candy,https://i5.walmartimages.com/asr/43cbcccb-8d77...,4.0


In [107]:
def hybrid_recommendations(train_data, target_user_id, item_name, top_n=10):
    content_based_rec = content_based_recommendations(train_data, item_name, top_n)
    collaborative_filtering_rec = collaborative_filtering_recommendations(train_data, target_user_id, top_n)
    
    hybrid_rec = pd.concat([content_based_rec, collaborative_filtering_rec]).drop_duplicates()
    return hybrid_rec.head(5)

In [108]:
target_user_id = 4
item_name = 'Black Radiance Perfect Tone Matte Lip Crème, Succulent Plum'

hybrid_rec = hybrid_recommendations(train_data,target_user_id, item_name, top_n=10)

print(f"Top 10 Hybrid Recommendations for User {target_user_id} and Item '{item_name}':")
hybrid_rec

C:\Users\gopik\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\gopik\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


Top 10 Hybrid Recommendations for User 4 and Item 'Black Radiance Perfect Tone Matte Lip Crème, Succulent Plum':


,Name,ReviewCount,Brand,ImageURL,Rating
896,"Black Radiance Perfect Tone Lip Color, Vintage...",78.0,Black Radiance,https://i5.walmartimages.com/asr/485f26b4-a19a...,4.7
2496,"Black Radiance Perfect Tone Lip Color, Hollywo...",18.0,Black Radiance,https://i5.walmartimages.com/asr/fe3da48f-5142...,4.3
2136,"Black Radiance Brilliant Effects Lip Gloss, Bl...",95.0,Black Radiance,https://i5.walmartimages.com/asr/f6784519-2c86...,4.4
3140,Laura Mercier Tone Perfecting Eye Gel Crème 0....,11.0,Laura Mercier,https://i5.walmartimages.com/asr/9cd9aea3-5ac3...,3.7
398,"IMAN Skin Tone Evener BB Crème, Light Sand",19.0,IMAN Cosmetics,https://i5.walmartimages.com/asr/b387123b-700d...,3.9


In [109]:
recommended_items = collaborative_filtering_rec['Name'].values.tolist()
recommended_item_ratings = []

for item_name in recommended_items:
    # Get the 'ProdID' of the item from the train_data where 'Name' matches the recommended item
    prod_id = train_data[train_data['Name'] == item_name]['ProdID'].values
    if len(prod_id) == 0:
        print(f"Product ID for '{item_name}' not found in train_data.")
        continue  # Skip if no product ID is found
    
    prod_id = prod_id[0]  # Assuming 'ProdID' is unique for each 'Name'
    
    # Check if prod_id is present in user_item_matrix
    if prod_id not in user_item_matrix.columns:
        print(f"Product {item_name} (ProdID: {prod_id}) not found in user-item matrix.")
        continue  # Skip if the product is not in the matrix

    # Get the rating of the target user for this item from the user_item_matrix
    if target_user_id in user_item_matrix.index:
        rating = user_item_matrix.loc[target_user_id, prod_id]  # Accessing rating based on target_user_id and prod_id
        recommended_item_ratings.append((item_name, rating))
    else:
        print(f"Target user {target_user_id} not found in user-item matrix.")
        continue

# Print the ratings of the recommended items
for item_name, rating in recommended_item_ratings:
    print(f"Item: {item_name}, Rating: {rating}")


Item: Nice n Easy Permanent Color, 111 Natural Medium Auburn 1 ea (Pack of 3), Rating: 0.0
Item: Old Spice Artisan Styling High Hold Matte Finish Molding Clay, 2.64 oz, Rating: 0.0
Item: Colgate My First Baby and Toddler Toothpaste, Fluoride Free and SLS Free, 1.75 Oz, Rating: 0.0
Item: (3 pack) Vaseline Lip Therapy Tinted Lip Balm Mini, Rosy 0.25 oz, Rating: 0.0
Item: The Art of Shaving Mens Hair Styling Gel, Juniper Scent, 2 Fl Oz, Rating: 0.0
Item: ORLY Nail Lacquer for Women, #20634 Red Carpet, 0.6 oz, Rating: 0.0
Item: LOreal Paris True Match Super-Blendable Blush, Soft Powder Texture, Soft Sun, 0.21 oz., Rating: 0.0
Item: Pantene Shampoo and Conditioner Set, Sheer Volume, 12-12.6 oz, Rating: 0.0
Item: Aromatherapy Essential Oil Diffuser 200ml - Magnifeko Cool Mist Humidifier Ultrasonic Aroma for room, Rating: 0.0
Item: Hard Candy Wet Ever Bold Hold Lip Lacquer Tin, 1212 Devious, Rating: 0.0


In [110]:
filtered_recommended_items = []

# Iterate through each recommended item
for item_name, rating in recommended_item_ratings:
    # Check if the rating is greater than 0
    if rating > 0:
        filtered_recommended_items.append((item_name, rating))

# Output the filtered recommended items
for item_name, rating in filtered_recommended_items:
    print(f"Item: {item_name}, Rating: {rating}")

In [111]:
precision = len(filtered_recommended_items)/len(collaborative_filtering_rec)

In [112]:
precision

0.0

In [113]:
recommended_items = hybrid_rec['Name'].values.tolist()
recommended_item_ratings = []

for item_name in recommended_items:
    # Get the 'ProdID' of the item from the train_data where 'Name' matches the recommended item
    prod_id = train_data[train_data['Name'] == item_name]['ProdID'].values
    if len(prod_id) == 0:
        print(f"Product ID for '{item_name}' not found in train_data.")
        continue  # Skip if no product ID is found
    
    prod_id = prod_id[0]  # Assuming 'ProdID' is unique for each 'Name'
    
    # Check if prod_id is present in user_item_matrix
    if prod_id not in user_item_matrix.columns:
        print(f"Product {item_name} (ProdID: {prod_id}) not found in user-item matrix.")
        continue  # Skip if the product is not in the matrix

    # Get the rating of the target user for this item from the user_item_matrix
    if target_user_id in user_item_matrix.index:
        rating = user_item_matrix.loc[target_user_id, prod_id]  # Accessing rating based on target_user_id and prod_id
        recommended_item_ratings.append((item_name, rating))
    else:
        print(f"Target user {target_user_id} not found in user-item matrix.")
        continue

# Print the ratings of the recommended items
for item_name, rating in recommended_item_ratings:
    print(f"Item: {item_name}, Rating: {rating}")

Item: Black Radiance Perfect Tone Lip Color, Vintage Plum, Rating: 4.7
Item: Black Radiance Perfect Tone Lip Color, Hollywood Glam, Rating: 0.4153846153846154
Item: Black Radiance Brilliant Effects Lip Gloss, Black Diamond, Rating: 1.9600000000000002
Item: Laura Mercier Tone Perfecting Eye Gel Crème 0.5 oz, Rating: 0.0
Item: IMAN Skin Tone Evener BB Crème, Light Sand, Rating: 1.55


In [114]:
filtered_recommended_items = []

# Iterate through each recommended item
for item_name, rating in recommended_item_ratings:
    # Check if the rating is greater than 0
    if rating > 0:
        filtered_recommended_items.append((item_name, rating))

# Output the filtered recommended items
for item_name, rating in filtered_recommended_items:
    print(f"Item: {item_name}, Rating: {rating}")

Item: Black Radiance Perfect Tone Lip Color, Vintage Plum, Rating: 4.7
Item: Black Radiance Perfect Tone Lip Color, Hollywood Glam, Rating: 0.4153846153846154
Item: Black Radiance Brilliant Effects Lip Gloss, Black Diamond, Rating: 1.9600000000000002
Item: IMAN Skin Tone Evener BB Crème, Light Sand, Rating: 1.55


In [115]:
precision = len(filtered_recommended_items)/len(hybrid_rec)

In [116]:
precision

0.8

In [117]:
recommended_items = content_based_rec['Name'].values.tolist()
recommended_item_ratings = []

for item_name in recommended_items:
    # Get the 'ProdID' of the item from the train_data where 'Name' matches the recommended item
    prod_id = train_data[train_data['Name'] == item_name]['ProdID'].values
    if len(prod_id) == 0:
        print(f"Product ID for '{item_name}' not found in train_data.")
        continue  # Skip if no product ID is found
    
    prod_id = prod_id[0]  # Assuming 'ProdID' is unique for each 'Name'
    
    # Check if prod_id is present in user_item_matrix
    if prod_id not in user_item_matrix.columns:
        print(f"Product {item_name} (ProdID: {prod_id}) not found in user-item matrix.")
        continue  # Skip if the product is not in the matrix

    # Get the rating of the target user for this item from the user_item_matrix
    if target_user_id in user_item_matrix.index:
        rating = user_item_matrix.loc[target_user_id, prod_id]  # Accessing rating based on target_user_id and prod_id
        recommended_item_ratings.append((item_name, rating))
    else:
        print(f"Target user {target_user_id} not found in user-item matrix.")
        continue

# Print the ratings of the recommended items
for item_name, rating in recommended_item_ratings:
    print(f"Item: {item_name}, Rating: {rating}")

Item: Kokie Professional Matte Lipstick, Kiss Me, 0.14 fl oz, Rating: 0.4153846153846154
Item: Kokie Professional Matte Lipstick, Firecracker, 0.14 fl oz, Rating: 0.0
Item: Kokie Professional Lip Poudre Liquid Matte Liquid Lipstick, Charmed, Rating: 2.7142857142857144
Item: Kokie Professional Lip Poudre Liquid Matte Liquid Lipstick, Infamous, Rating: 0.0
Item: Be Matte Lipstick - Pink, Rating: 1.942857142857143
Item: Kokie Professional Nail Polish, Girls Night , 0.54 fl oz, Rating: 0.0
Item: Kokie Professional Nail Polish, Jamacan Me Blush, 0.54 fl oz, Rating: 3.8200000000000003
Item: Kokie Professional Skin Perfect HD Foundation, 90C, 1.0 fl oz, Rating: 0.0


In [118]:
filtered_recommended_items = []

# Iterate through each recommended item
for item_name, rating in recommended_item_ratings:
    # Check if the rating is greater than 0
    if rating > 0:
        filtered_recommended_items.append((item_name, rating))

# Output the filtered recommended items
for item_name, rating in filtered_recommended_items:
    print(f"Item: {item_name}, Rating: {rating}")

Item: Kokie Professional Matte Lipstick, Kiss Me, 0.14 fl oz, Rating: 0.4153846153846154
Item: Kokie Professional Lip Poudre Liquid Matte Liquid Lipstick, Charmed, Rating: 2.7142857142857144
Item: Be Matte Lipstick - Pink, Rating: 1.942857142857143
Item: Kokie Professional Nail Polish, Jamacan Me Blush, 0.54 fl oz, Rating: 3.8200000000000003


In [119]:
precision = len(filtered_recommended_items)/len(content_based_rec)

In [120]:
precision

0.5